In [3]:
import pandas as pd
from wsi.config import INDICATORS
from wsi.utils import processed_data_path

df = pd.read_csv(processed_data_path("womens_safety_index.csv"))
indicator_columns = [col for col in df.columns if col in INDICATORS]

missing_records = []
for col in indicator_columns:
    missing_rows = df[df[col].isnull()][['ISO_code', 'Economy', 'Region']].copy()
    if not missing_rows.empty:
        missing_rows['Indicator'] = col
        missing_records.append(missing_rows)

# Combine and deduplicate
if missing_records:
    all_missing = pd.concat(missing_records, ignore_index=True)
    unique_missing = all_missing.drop_duplicates().reset_index(drop=True)
    display(unique_missing)  # Use display() in Jupyter for cleaner output
else:
    print("No missing values found.")


,ISO_code,Economy,Region,Indicator
0,AGO,Angola,Central Africa,Attitudes Towards Violence
1,CAF,Central African Republic,Central Africa,Attitudes Towards Violence
2,CMR,Cameroon,Central Africa,Attitudes Towards Violence
3,COD,"Congo, Dem. Rep.",Central Africa,Attitudes Towards Violence
4,COG,"Congo, Rep.",Central Africa,Attitudes Towards Violence
5,FJI,Fiji,Melanesia,Attitudes Towards Violence
6,FSM,Micronesia,Micronesia,Attitudes Towards Violence
7,GAB,Gabon,Central Africa,Attitudes Towards Violence
8,GNQ,Equatorial Guinea,Middle Africa,Attitudes Towards Violence
9,KIR,Kiribati,Micronesia,Attitudes Towards Violence
